 # Libraries Installation

### OS Issues

* EPANET Simulator under wntr doesn't work in MacOS, Linux but does in Windows
* Windows has (.dll) precompiled but MacOS needs it to be compiled (.dylib)
* Working on compiling .dylib and .so for MacOS and Linux

In [23]:
# MacOS specific .dylib 
from pathlib import Path
import os
import platform

system_os = platform.system()

# Get current script directory
base_path = Path().resolve()

if system_os == "Darwin":  # macOS
    epanet_lib_path = base_path / "lib" / "libepanet.dylib"
    os.environ["EPANET_PATH"] = str(epanet_lib_path)

elif system_os == "Linux":
    epanet_lib_path = base_path / "lib" / "libepanet.so"
    os.environ["EPANET_PATH"] = str(epanet_lib_path)

elif system_os == "Windows":
    print("Windows detected. No need to set EPANET_PATH.")

else:
    raise EnvironmentError(f"Unsupported OS: {system_os}")



In [24]:
# Upgrade pip and install packages from requirements.txt (cross-platform safe)
%pip install --upgrade pip
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Genetic Algorithm

## Setting up Manual Functions

### Importing Libraries

In [25]:
import wntr
import numpy as np
from datetime import timedelta
import pandas as pd
import random
import copy

### Auxiliary Functions

In [26]:
# --- RESHAPE INDIVIDUAL TO SCHEDULE ---

def reshape_individual(individual, wntk):

    # Reshape the individual to a 2D array
    schedule = np.array(individual).reshape(wntk.describe(level=1)['Links']['Pumps'], (int(wntk.options.time.duration / wntk.options.time.hydraulic_timestep) + 1)).tolist()

    return schedule

In [27]:
# --- SCHEDULE ADDITION ---

def add_schedule(schedule, wntk):
    for i in range(wntk.describe(level=1)['Links']['Pumps']):
        for j in range(int(wntk.options.time.duration / wntk.options.time.hydraulic_timestep) + 1):
            pump = wntk.get_link(wntk.pump_name_list[i])
            condition = wntr.network.controls.SimTimeCondition(wntk, '=', str(timedelta(hours=j)))
            action = wntr.network.controls.ControlAction(pump, 'status', schedule[i][j])
            control = wntr.network.controls.Control(condition, action, name=f'Control_pump{i}_time{j}')
            wntk.add_control(f"Control Pump ID : {i}, Hour : {j}", control)

In [28]:
# --- ADDED SCHEDULE REMOVAL {EXCLUSIVELY THE ONES FROM ADD SCHEDULE}---

def remove_schedule(wntk):
    for i in range(wntk.describe(level=1)['Links']['Pumps']):
        for j in range(int(wntk.options.time.duration / wntk.options.time.hydraulic_timestep) + 1):
            wntk.remove_control(f"Control Pump ID : {i}, Hour : {j}")

In [ ]:
# --- COST OBJECTIVE FUNCTION ---

def cost_objective(wntk, cost_pattern = np.array([0.7]*6 + [1.2]*12 + [1]*7).T): # Cost pattern needs to be changed as per Time requirement (size must be equal to the number of time steps + 1)
    # Running Epanet simulation
    simulation = wntr.sim.EpanetSimulator(wntk)
    results = simulation.run_sim()

    # Obtaining cost objective
    # cost_pattern = 
    base_cost = 5 # Base cost per hour
    headloss_df = results.link['headloss'][wntk.pump_name_list]
    cost_df = -1 * base_cost * headloss_df.multiply(cost_pattern, axis=0)

    return cost_df

In [ ]:
# --- DEMAND OBJECTIVE FUNCTION ---

def demand_objective(wntk, req_pressure = 10):
    # NOTE : The required_pressure, minimum_pressure, pressure_exponent can be applied separately for each node (refer to the WNTR documentation hydraulics.rst)
    # For sake of simplicity, we are using the same values for all nodes

    # Setting up Pressure Driven Demand model
    wntk.options.hydraulic.demand_model = 'PDD'
    wntk.options.hydraulic.required_pressure = req_pressure
    
    # Running WNTR simulation
    simulation = wntr.sim.WNTRSimulator(wntk)
    results = simulation.run_sim()

    # Obtaining demand objective
    expected_demand = wntr.metrics.expected_demand(wntk)
    demand = results.node['demand'].loc[:,wntk.junction_name_list]    
    wsa = wntr.metrics.water_service_availability(expected_demand, demand)

    return wsa

## Setting up DEAP framework

### Setting up Libraries

In [31]:
# Necessary Libraries for DEAP and Torch
from deap import base, tools, creator, algorithms

# --- CONFIGURATION ---
wn = wntr.network.WaterNetworkModel('Network Files/Net3.inp')
NUM_PUMPS = wn.describe(level=1)['Links']['Pumps']
NUM_TIMESTAMPS = (int(wn.options.time.duration / wn.options.time.hydraulic_timestep) + 1)
CHROM_SIZE = NUM_PUMPS * NUM_TIMESTAMPS  # No of Pumps * No of Timestamps

### Evaluation Function

In [ ]:
# --- EVALUATION FUNCTION ---

def evaluation(individual, wntk = wn):

    wntk_copy = copy.deepcopy(wntk)

    # Reshape the individual to a schedule
    schedule = reshape_individual(individual, wntk_copy)

    # Add the schedule to the WNTR model
    add_schedule(schedule, wntk_copy)

    # Get Cost Objective - NOTE !!! Price pattern is defined in cost objective function and can be changed inside the cost objective function if necessary
    cost_df = cost_objective(wntk_copy)
    total_cost = cost_df.sum().sum()

    # Get Demand Objective
    wsa = demand_objective(wntk_copy)
    wsd = 1 - wsa.where(pd.isna(wsa), wsa) # water service deficit
    total_demand = np.nansum(wsd.to_numpy())

    # Reset the simulation
    remove_schedule(wntk_copy)
    wntk_copy.reset_initial_values()

    return (total_cost + (total_demand*100), )

In [ ]:
# --- CREATE TYPES ---

# Assigning Mult-Class Fitness Weigths
creator.create("FitnessMulti", base.Fitness, weights=(-1.0,))

# Defining Individual
creator.create("Individual", list, fitness = creator.FitnessMulti)

toolbox = base.Toolbox()

# --- CREATE TOOLS ---

# Define Binary Genes
toolbox.register("attr_bool", random.randint, 0, 1)

# Define Individual (List of Binary Values)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=CHROM_SIZE)

# Define Random Population
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Register the Fitness function
toolbox.register("evaluate", evaluation)

# Define Selection Methodology
toolbox.register("select", tools.selTournament, tournsize = 3)

# Define Crossover Methodology
toolbox.register("mate", tools.cxTwoPoint)

# Define Mutation Methodology
toolbox.register("mutate", tools.mutFlipBit, indpb = 0.1)

# Set Hall of Fame
hof = tools.HallOfFame(10)

In [ ]:
# --- GENETIC ALGO LOOP---

def run_ga(pop_size = 40, generations = 50, cx_prob = 0.7, mut_prob = 0.3):
    #Create
    population = toolbox.population(n = pop_size)

    # Objective Statistics
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("min", np.min)
    stats.register("max", np.max)

    # Run the Genetic Algorithm
    population, logbook = algorithms.eaSimple(
        population=population,
        toolbox=toolbox,
        cxpb=cx_prob,
        mutpb=mut_prob,
        ngen=generations,
        stats=stats,
        halloffame=hof,
        verbose=True,
    )
    
    best_ind = hof[0]
    print("Best Individual : ", best_ind)
    print("Fitness : ", best_ind.fitness.values)

    return best_ind, logbook, population, hof

In [ ]:
best_sol, logbook, population, hof = run_ga(40, 5)

# Test Runs

# Solution Analysis

In [ ]:
# Enter the individual value to Analyze
analysis_individual = best_sol
print(best_sol)

## Analysis Functions

In [ ]:
# --- COST ANALYSIS FUNCTION ---

def cost_analysis(analysis_individual, cost_pattern = np.array([0.7]*6 + [1.2]*12 + [1]*7).T, wntk=wn): # Cost pattern needs to be changed as per Time requirement (size must be equal to the number of time steps + 1)
    
    wntk_copy = copy.deepcopy(wntk)

    # Reshape the individual to a schedule
    schedule = reshape_individual(analysis_individual, wntk_copy)

    # Add the schedule to the WNTR model
    add_schedule(schedule, wntk_copy)
    
    # Get Cost Objective - NOTE !!! Price pattern is defined in cost objective function and can be changed inside the cost objective function if necessary
    cost_df = cost_objective(wntk_copy)

    # Analysis Plots


    # Reset the simulation
    remove_schedule(wntk_copy)
    wntk_copy.reset_initial_values()
    
    return cost_df

In [ ]:
# --- DEMAND ANALYSIS FUNCTION ---

def demand_analysis(analysis_individual, wntk=wn):
    
    wntk_copy = copy.deepcopy(wntk)

    # Reshape the individual to a schedule
    schedule = reshape_individual(analysis_individual, wntk_copy)

    # Add the schedule to the WNTR model
    add_schedule(schedule, wntk_copy)

    # Get Demand Objective
    wsa = demand_objective(wntk_copy)

    # Analysis Plots


    # Reset the simulation
    remove_schedule(wntk_copy)
    wntk_copy.reset_initial_values()
    
    return wsa

## Cost Analysis

In [ ]:
cost_analysis(analysis_individual)

## Demand Analysis

In [ ]:
demand_analysis(analysis_individual)